🎯 Nivel 1: Básico (Entender el código)

Ejercicio 1. 1: Explorar el Dataset

📋 OBJETIVO: Familiarizarse con los datos

TAREAS:
1. Carga el archivo failures_for_embeddings.csv
2. ¿Cuántas startups hay?
3. ¿Cuántos sectores únicos existen?
4.  ¿Cuál es el sector con más startups fallidas?
5.  Muestra 5 ejemplos de startups del sector "Tech"

ENTREGA: Notebook con respuestas y código

In [ ]:
import pandas as pd

# Cargar datos
df = pd.read_csv("data/gold/failures_for_embeddings.csv")

# 1. ¿Cuántas startups hay?
print(f"Total startups: {len(df)}")
# Respuesta: 409

# 2. ¿Cuántos sectores únicos? 
print(f"Sectores únicos: {df['sector'].nunique()}")
# Respuesta: 12 (aproximadamente)

# 3. ¿Sector con más startups fallidas?
print("Sector más común:")
print(df['sector'].value_counts().head(1))
# Respuesta: Probablemente "Information" o "Tech"

# 4. Distribución completa de sectores
print("\nDistribución por sector:")
print(df['sector'].value_counts())

# 5. 5 ejemplos del sector Tech
print("\n5 startups de Tech:")
tech_startups = df[df['sector']. str.contains('Tech|Information', case=False, na=False)]
print(tech_startups[['name', 'sector', 'text']].head(5))


Ejercicio 1.2: Entender los Embeddings

📋 OBJETIVO: Comprender qué son los embeddings

TAREAS:
1. Carga el archivo embeddings. npy
2. ¿Qué dimensiones tiene la matriz?
3.  ¿Qué representa cada fila?
4.  ¿Qué representa cada columna? 
5.  Muestra el embedding de la primera startup

PREGUNTAS:
- ¿Por qué son 384 dimensiones? 
- ¿Qué modelo generó estos embeddings? 

ENTREGA: Respuestas + código

In [ ]:
import numpy as np
import pandas as pd

# Cargar embeddings
embeddings = np.load("data/gold/embeddings.npy")
df = pd.read_csv("data/gold/failures_for_embeddings.csv")

# 1. Dimensiones de la matriz
print(f"Shape: {embeddings.shape}")
# Respuesta: (409, 384)

# 2. ¿Qué representa cada fila?
print(f"\nCada fila = 1 startup")
print(f"Total filas: {embeddings.shape[0]} startups")

# 3. ¿Qué representa cada columna? 
print(f"\nCada columna = 1 dimensión del embedding")
print(f"Total columnas: {embeddings.shape[1]} dimensiones")

# 4. Embedding de la primera startup
print(f"\nPrimera startup: {df.iloc[0]['name']}")
print(f"Embedding (primeros 10 valores): {embeddings[0][:10]}")
print(f"Embedding (últimos 10 valores): {embeddings[0][-10:]}")

# 5. Estadísticas del embedding
print(f"\nEstadísticas del primer embedding:")
print(f"  Min: {embeddings[0].min():. 4f}")
print(f"  Max: {embeddings[0].max():.4f}")
print(f"  Media: {embeddings[0].mean():. 4f}")
print(f"  Std: {embeddings[0].std():.4f}")

# RESPUESTAS A PREGUNTAS:
print("\n" + "="*50)
print("RESPUESTAS:")
print("="*50)
print("""
¿Por qué 384 dimensiones? 
- Es la arquitectura del modelo all-MiniLM-L6-v2
- Modelos más grandes tienen más dimensiones (768, 1024)
- 384 es un buen balance entre calidad y velocidad

¿Qué modelo generó estos embeddings? 
- Sentence Transformers: all-MiniLM-L6-v2
- Es un modelo pre-entrenado para generar embeddings de oraciones
- Basado en BERT pero más pequeño y rápido
""")


Ejercicio 1.3: Cosine Similarity Manual

📋 OBJETIVO: Calcular similitud entre dos textos

TAREAS:
1.  Usa Sentence Transformers para generar embeddings de:
   - "App de delivery de comida"
   - "Plataforma de entrega de alimentos"
   - "Software de contabilidad"
   
2. Calcula la similitud coseno entre los 3 pares
3. ¿Cuáles son más similares?  ¿Por qué?

ENTREGA: Código + explicación

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Cargar modelo
model = SentenceTransformer('all-MiniLM-L6-v2')

# Textos a comparar
textos = [
    "App de delivery de comida",           # Texto 0
    "Plataforma de entrega de alimentos",  # Texto 1
    "Software de contabilidad"             # Texto 2
]

# Generar embeddings
embeddings = model.encode(textos)
print(f"Shape de embeddings: {embeddings.shape}")  # (3, 384)

# Calcular similitud coseno entre todos los pares
similitudes = cosine_similarity(embeddings)

print("\nMatriz de similitud:")
print("="*50)
for i, texto_i in enumerate(textos):
    for j, texto_j in enumerate(textos):
        print(f"[{i}] vs [{j}]: {similitudes[i][j]:.4f}")
    print()

# Interpretación más clara
print("\nRESULTADOS:")
print("="*50)
print(f"'Delivery de comida' vs 'Entrega de alimentos': {similitudes[0][1]:.4f}")
print(f"'Delivery de comida' vs 'Software contabilidad': {similitudes[0][2]:.4f}")
print(f"'Entrega de alimentos' vs 'Software contabilidad': {similitudes[1][2]:.4f}")

print("\nCONCLUSIÓN:")
print("="*50)
print("""
- Los textos 0 y 1 son MUY SIMILARES (~0.75-0.85)
  Aunque usan palabras diferentes, significan lo mismo
  
- Los textos 0/1 vs 2 son MUY DIFERENTES (~0.15-0.25)
  Son dominios completamente distintos

- Esto demuestra que los embeddings capturan SIGNIFICADO,
  no solo coincidencia de palabras
""")

# BONUS: Visualizar como heatmap
import matplotlib. pyplot as plt

plt.figure(figsize=(8, 6))
plt.imshow(similitudes, cmap='RdYlGn', vmin=0, vmax=1)
plt.colorbar(label='Similitud')
plt. xticks([0, 1, 2], ['Delivery', 'Entrega', 'Contabilidad'], rotation=45)
plt.yticks([0, 1, 2], ['Delivery', 'Entrega', 'Contabilidad'])
plt.title('Matriz de Similitud Coseno')
for i in range(3):
    for j in range(3):
        plt.text(j, i, f'{similitudes[i][j]:. 2f}', ha='center', va='center')
plt.tight_layout()
plt.savefig('similitud_heatmap. png')
plt.show()

🎯 Nivel 2: Intermedio (Modificar el código)

Ejercicio 2. 1: Cambiar el Top-K

📋 OBJETIVO: Modificar cuántos resultados devuelve la búsqueda

TAREAS:
1. Modifica la función buscar_startups_similares()
2. Añade un parámetro top_k con valor por defecto 3
3. Prueba con top_k = 1, 3, 5, 10
4.  ¿Cómo afecta al resultado?

BONUS: Añade un slider en Streamlit para que el usuario elija

ENTREGA: Código modificado + observaciones

In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics. pairwise import cosine_similarity

# Cargar datos
df = pd. read_csv("data/gold/failures_for_embeddings. csv")
embeddings = np. load("data/gold/embeddings.npy")
model = SentenceTransformer('all-MiniLM-L6-v2')

def buscar_startups_similares(descripcion, top_k=3):
    """
    Busca las startups más similares a la descripción dada. 
    
    Args:
        descripcion: Texto describiendo la startup
        top_k: Número de resultados a devolver (default: 3)
    
    Returns:
        DataFrame con las startups más similares
    """
    # 1. Generar embedding del usuario
    embedding_usuario = model.encode([descripcion])
    
    # 2. Calcular similitud con todos
    similitudes = cosine_similarity(embedding_usuario, embeddings)[0]
    
    # 3.  Obtener índices de los top_k más similares
    top_indices = np. argsort(similitudes)[-top_k:][::-1]
    
    # 4.  Crear dataframe con resultados
    resultados = df.iloc[top_indices]. copy()
    resultados['similitud'] = similitudes[top_indices]
    
    return resultados[['name', 'sector', 'text', 'similitud']]


# PRUEBAS con diferentes top_k
query = "food delivery app with high costs"

print("="*60)
print(f"Query: '{query}'")
print("="*60)

for k in [1, 3, 5, 10]:
    print(f"\n--- TOP {k} ---")
    resultados = buscar_startups_similares(query, top_k=k)
    for idx, row in resultados.iterrows():
        print(f"  {row['name']:20} | Sim: {row['similitud']:.4f} | {row['sector']}")

# OBSERVACIONES
print("\n" + "="*60)
print("OBSERVACIONES:")
print("="*60)
print("""
- top_k=1: Solo el más similar, útil para respuestas rápidas
- top_k=3: Balance entre diversidad y relevancia (recomendado)
- top_k=5: Más contexto para el LLM
- top_k=10: Demasiado, puede incluir resultados poco relevantes

La similitud baja conforme aumenta K:
- Top 1: ~0.40
- Top 3: ~0.35-0.40
- Top 5: ~0.30-0.40
- Top 10: ~0.25-0.40

RECOMENDACIÓN: top_k=3 o top_k=5 para RAG
""")


# BONUS: Slider en Streamlit
"""
import streamlit as st

st.title("Startup Death Oracle")

descripcion = st.text_area("Describe tu startup:")

# Slider para top_k
top_k = st.slider("¿Cuántas startups similares buscar?", 1, 10, 3)

if st.button("Predecir"):
    resultados = buscar_startups_similares(descripcion, top_k=top_k)
    st.dataframe(resultados)
"""

Ejercicio 2.2: Añadir Filtro por Sector

📋 OBJETIVO: Filtrar resultados por sector

TAREAS:
1. Modifica la búsqueda para aceptar un parámetro sector
2. Si sector != None, solo buscar en ese sector
3.  Prueba: buscar "delivery app" solo en sector "Food"

PREGUNTAS:
- ¿Cambian los resultados? 
- ¿Cuándo sería útil este filtro? 

ENTREGA: Código + pruebas

In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn. metrics.pairwise import cosine_similarity

# Cargar datos
df = pd.read_csv("data/gold/failures_for_embeddings.csv")
embeddings = np.load("data/gold/embeddings.npy")
model = SentenceTransformer('all-MiniLM-L6-v2')

def buscar_startups_similares(descripcion, top_k=3, sector=None):
    """
    Busca startups similares, opcionalmente filtradas por sector. 
    
    Args:
        descripcion: Texto describiendo la startup
        top_k: Número de resultados
        sector: Filtrar por sector (None = todos)
    
    Returns:
        DataFrame con resultados
    """
    # 1.  Generar embedding del usuario
    embedding_usuario = model.encode([descripcion])
    
    # 2. Filtrar por sector si se especifica
    if sector:
        # Obtener índices del sector
        mask = df['sector']. str.contains(sector, case=False, na=False)
        df_filtrado = df[mask]. copy()
        embeddings_filtrados = embeddings[mask]
        
        if len(df_filtrado) == 0:
            print(f"No hay startups en el sector '{sector}'")
            return pd.DataFrame()
    else:
        df_filtrado = df.copy()
        embeddings_filtrados = embeddings
    
    # 3. Calcular similitud
    similitudes = cosine_similarity(embedding_usuario, embeddings_filtrados)[0]
    
    # 4.  Obtener top_k (o menos si no hay suficientes)
    k = min(top_k, len(df_filtrado))
    top_indices = np.argsort(similitudes)[-k:][::-1]
    
    # 5.  Crear resultados
    resultados = df_filtrado.iloc[top_indices].copy()
    resultados['similitud'] = similitudes[top_indices]
    
    return resultados[['name', 'sector', 'text', 'similitud']]


# PRUEBAS
query = "food delivery app"

print("="*60)
print(f"Query: '{query}'")
print("="*60)

# Sin filtro
print("\n--- SIN FILTRO (todos los sectores) ---")
resultados = buscar_startups_similares(query, top_k=5)
for idx, row in resultados. iterrows():
    print(f"  {row['name']:20} | {row['sector']:30} | {row['similitud']:.4f}")

# Con filtro Food
print("\n--- FILTRO: Food ---")
resultados = buscar_startups_similares(query, top_k=5, sector="Food")
for idx, row in resultados.iterrows():
    print(f"  {row['name']:20} | {row['sector']:30} | {row['similitud']:.4f}")

# Con filtro Health
print("\n--- FILTRO: Health ---")
resultados = buscar_startups_similares(query, top_k=5, sector="Health")
for idx, row in resultados.iterrows():
    print(f"  {row['name']:20} | {row['sector']:30} | {row['similitud']:.4f}")

# Listar sectores disponibles
print("\n--- SECTORES DISPONIBLES ---")
print(df['sector'].value_counts())

# OBSERVACIONES
print("\n" + "="*60)
print("OBSERVACIONES:")
print("="*60)
print("""
¿Cambian los resultados? 
- SÍ, significativamente
- Sin filtro: mezcla sectores (Food, Tech, Health)
- Con filtro Food: solo startups de comida
- Con filtro Health: busca "delivery" en contexto de salud

¿Cuándo es útil este filtro?
- Cuando el usuario quiere compararse con su industria
- Para análisis específico de un sector
- Para evitar comparaciones irrelevantes
  (ej: una startup de food no debería compararse con fintech)

CUIDADO:
- Si el sector tiene pocas startups, los resultados serán pobres
- Algunos sectores tienen <10 startups
""")

Ejercicio 2. 3: Nuevo Modelo de Embeddings

📋 OBJETIVO: Comparar diferentes modelos de embeddings

TAREAS:
1.  Genera embeddings con estos 3 modelos:
   - all-MiniLM-L6-v2 (actual)
   - all-mpnet-base-v2
   - paraphrase-multilingual-MiniLM-L12-v2

2. Para cada modelo:
   - ¿Cuántas dimensiones tiene? 
   - ¿Cuánto tarda en generar 409 embeddings? 
   
3. Busca "food delivery app" con cada modelo
4. ¿Cambian los resultados?  ¿Cuál es mejor?

ENTREGA: Tabla comparativa + conclusiones

In [ ]:
import time
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar datos
df = pd.read_csv("data/gold/failures_for_embeddings.csv")
textos = df['text'].tolist()

# Modelos a comparar
modelos = [
    'all-MiniLM-L6-v2',                    # Actual (rápido, ligero)
    'all-mpnet-base-v2',                   # Mejor calidad, más lento
    'paraphrase-multilingual-MiniLM-L12-v2'  # Multilingüe
]

resultados_modelos = {}

print("="*70)
print("COMPARACIÓN DE MODELOS DE EMBEDDINGS")
print("="*70)

for nombre_modelo in modelos:
    print(f"\n--- Cargando: {nombre_modelo} ---")
    
    # Cargar modelo
    model = SentenceTransformer(nombre_modelo)
    
    # Medir tiempo de generación
    inicio = time.time()
    embeddings = model.encode(textos, show_progress_bar=True)
    tiempo = time.time() - inicio
    
    # Guardar resultados
    resultados_modelos[nombre_modelo] = {
        'embeddings': embeddings,
        'dimensiones': embeddings.shape[1],
        'tiempo': tiempo,
        'model': model
    }
    
    print(f"  Dimensiones: {embeddings.shape[1]}")
    print(f"  Tiempo: {tiempo:.2f} segundos")
    print(f"  Tamaño: {embeddings.nbytes / 1024 / 1024:.2f} MB")


# TABLA COMPARATIVA
print("\n" + "="*70)
print("TABLA COMPARATIVA")
print("="*70)
print(f"{'Modelo':<45} | {'Dims':>6} | {'Tiempo':>8} | {'MB':>6}")
print("-"*70)
for nombre, datos in resultados_modelos.items():
    mb = datos['embeddings'].nbytes / 1024 / 1024
    print(f"{nombre:<45} | {datos['dimensiones']:>6} | {datos['tiempo']:>6.2f}s | {mb:>5.2f}")


# COMPARAR BÚSQUEDA
query = "food delivery app with high costs"
print("\n" + "="*70)
print(f"BÚSQUEDA: '{query}'")
print("="*70)

for nombre_modelo, datos in resultados_modelos.items():
    print(f"\n--- {nombre_modelo} ---")
    
    # Generar embedding de la query
    query_embedding = datos['model'].encode([query])
    
    # Calcular similitud
    similitudes = cosine_similarity(query_embedding, datos['embeddings'])[0]
    
    # Top 3
    top_indices = np.argsort(similitudes)[-3:][::-1]
    
    for i, idx in enumerate(top_indices):
        print(f"  {i+1}. {df.iloc[idx]['name']:20} | Sim: {similitudes[idx]:.4f}")


# CONCLUSIONES
print("\n" + "="*70)
print("CONCLUSIONES")
print("="*70)
print("""
┌─────────────────────────────────────────────────────────────────────┐
│ MODELO                              │ PROS              │ CONTRAS   │
├─────────────────────────────────────┼───────────────────┼───────────┤
│ all-MiniLM-L6-v2                    │ Rápido, ligero    │ Menos     │
│ (ACTUAL)                            │ 384 dims          │ preciso   │
├─────────────────────────────────────┼───────────────────┼───────────┤
│ all-mpnet-base-v2                   │ Mejor calidad     │ Más lento │
│                                     │ 768 dims          │ Más RAM   │
├─────────────────────────────────────┼───────────────────┼───────────┤
│ paraphrase-multilingual-MiniLM-L12  │ Multilingüe       │ Menos     │
│                                     │ Español nativo    │ preciso   │
│                                     │                   │ en inglés │
└─────────────────────────────────────┴───────────────────┴───────────┘

RECOMENDACIONES:
- Prototipo rápido: all-MiniLM-L6-v2 ✓ (actual)
- Mejor calidad: all-mpnet-base-v2
- Datos en español: paraphrase-multilingual-MiniLM-L12-v2
- Producción con mucho tráfico: all-MiniLM-L6-v2

¿Cambian los resultados? 
- Sí, ligeramente
- Los top-1 suelen coincidir
- Los top-3 pueden variar en orden
- mpnet tiende a dar scores más altos
""")

Ejercicio 2.4: Implementar ChromaDB desde Cero

📋 OBJETIVO: Crear tu propia colección en ChromaDB

TAREAS:
1. Instala chromadb: pip install chromadb
2. Crea una colección llamada "mis_startups"
3. Añade las 409 startups con sus metadatos
4.  Implementa función buscar_con_mi_chroma()
5.  Compara resultados con Cosine manual

ENTREGA: Código completo funcionando

In [ ]:
import chromadb
from chromadb. utils import embedding_functions
import pandas as pd

# Cargar datos
df = pd.read_csv("data/gold/failures_for_embeddings.csv")

print("="*60)
print("IMPLEMENTACIÓN DE CHROMADB")
print("="*60)

# 1. Crear cliente de ChromaDB
print("\n1.  Creando cliente ChromaDB...")
client = chromadb. Client()  # En memoria
# client = chromadb.PersistentClient(path="./chroma_db")  # Persistente

# 2. Configurar función de embeddings
print("2. Configurando embedding function...")
sentence_transformer_ef = embedding_functions. SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# 3. Crear colección (eliminar si existe)
print("3.  Creando colección 'mis_startups'...")
try:
    client.delete_collection("mis_startups")
except:
    pass

collection = client.create_collection(
    name="mis_startups",
    embedding_function=sentence_transformer_ef,
    metadata={"description": "Startups fallidas para RAG"}
)

# 4.  Preparar datos para insertar
print("4.  Preparando datos...")
documents = df['text']. tolist()
metadatas = [
    {
        "nombre": row['name'],
        "sector": row['sector'] if pd.notna(row['sector']) else "Unknown"
    }
    for _, row in df.iterrows()
]
ids = [f"startup_{i}" for i in range(len(df))]

# 5. Añadir documentos a la colección
print("5.  Añadiendo documentos a ChromaDB...")
# Añadir en batches para evitar problemas de memoria
batch_size = 100
for i in range(0, len(documents), batch_size):
    end = min(i + batch_size, len(documents))
    collection.add(
        documents=documents[i:end],
        metadatas=metadatas[i:end],
        ids=ids[i:end]
    )
    print(f"   Añadidos {end}/{len(documents)} documentos")

print(f"\n✓ Colección creada con {collection.count()} documentos")


# 6.  Función de búsqueda con ChromaDB
def buscar_con_mi_chroma(query, n_results=3, sector=None):
    """
    Busca startups similares usando ChromaDB. 
    
    Args:
        query: Texto de búsqueda
        n_results: Número de resultados
        sector: Filtrar por sector (opcional)
    
    Returns:
        dict con resultados
    """
    # Preparar filtro si hay sector
    where_filter = None
    if sector:
        where_filter = {"sector": {"$contains": sector}}
    
    # Buscar
    resultados = collection.query(
        query_texts=[query],
        n_results=n_results,
        where=where_filter,
        include=["documents", "metadatas", "distances"]
    )
    
    return resultados


# 7.  PRUEBAS
print("\n" + "="*60)
print("PRUEBAS DE BÚSQUEDA")
print("="*60)

# Prueba 1: Búsqueda simple
query = "food delivery app with high costs"
print(f"\nQuery: '{query}'")
print("-"*40)

resultados = buscar_con_mi_chroma(query, n_results=3)

for i in range(len(resultados['ids'][0])):
    nombre = resultados['metadatas'][0][i]['nombre']
    sector = resultados['metadatas'][0][i]['sector']
    distancia = resultados['distances'][0][i]
    similitud = 1 - distancia  # ChromaDB usa distancia, no similitud
    print(f"{i+1}.  {nombre:20} | {sector:25} | Sim: {similitud:. 4f}")


# Prueba 2: Con filtro de sector
print(f"\nQuery con filtro sector='Food':")
print("-"*40)

resultados = buscar_con_mi_chroma(query, n_results=3, sector="Food")

if resultados['ids'][0]:
    for i in range(len(resultados['ids'][0])):
        nombre = resultados['metadatas'][0][i]['nombre']
        sector = resultados['metadatas'][0][i]['sector']
        distancia = resultados['distances'][0][i]
        similitud = 1 - distancia
        print(f"{i+1}. {nombre:20} | {sector:25} | Sim: {similitud:.4f}")
else:
    print("No se encontraron resultados con ese filtro")


# 8. COMPARACIÓN CON MÉTODO MANUAL
print("\n" + "="*60)
print("COMPARACIÓN: ChromaDB vs Cosine Manual")
print("="*60)

import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics. pairwise import cosine_similarity
import time

# Cargar para método manual
embeddings_np = np.load("data/gold/embeddings.npy")
model = SentenceTransformer('all-MiniLM-L6-v2')

# Medir tiempo ChromaDB
inicio = time.time()
for _ in range(10):
    buscar_con_mi_chroma(query, n_results=3)
tiempo_chroma = (time.time() - inicio) / 10

# Medir tiempo Cosine manual
inicio = time. time()
for _ in range(10):
    query_emb = model.encode([query])
    sims = cosine_similarity(query_emb, embeddings_np)
    top_3 = np.argsort(sims[0])[-3:][::-1]
tiempo_manual = (time.time() - inicio) / 10

print(f"ChromaDB: {tiempo_chroma*1000:.2f} ms por búsqueda")
print(f"Cosine Manual: {tiempo_manual*1000:.2f} ms por búsqueda")
print(f"ChromaDB es {tiempo_manual/tiempo_chroma:. 1f}x más rápido")

print("""
NOTA: Con 409 documentos la diferencia es pequeña. 
Con 1,000,000 documentos, ChromaDB sería ~1000x más rápido.
""")
